In [1]:
import sdf
import numpy as np

In [2]:
# Create the data arrays:
t = np.linspace(0, 10, 51)
v = np.sin(t)

In [3]:
# Create the datasets:
ds_t = sdf.Dataset('t', data=t, unit='s', is_scale=True, display_name='Time')
ds_v = sdf.Dataset('v', data=v, unit='V', scales=[ds_t])

In [4]:
# Create the root group and write the file:
g = sdf.Group('/', comment='A sine voltage', datasets=[ds_t, ds_v])
sdf.save('sine.sdf', g)

In [ ]:
# Read the dataset from the SDF file asserting the correct unit of the dataset and scale:
ds_v2 = sdf.load('sine.sdf', '/v', unit='V', scale_units=['s'])

In [6]:
# Get the meta info and data array from the dataset:
print(ds_v2.unit)
print(ds_v2.data.shape)

V
(51,)


In [7]:
# Get the scale for the first dimension:
ds_t2 = ds_v2.scales[0]
print(ds_t2.unit)

s


In [8]:
# Load 2006 LUT for interpolation based on iteration

# Pressure range [MPa] from 2006 LUT, convert to [Pa]
P = np.array((0.10,0.30,0.50,1.0,2.0,3.0,5.0,7.0,10.0,12.0,14.0,16.0,18.0,20.0,21.0))*1e6

# Mass Flux range [kg/m^2-s] from 2006 LUT
G = np.array((0,50,100,300,500,750,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000))

# Quality range from 2006 LUT
x = np.array((-0.50,-0.40,-0.30,-0.20,-0.15,-0.10,-0.05,0.00,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.60,0.70,0.80,0.90,1.00))

In [9]:
# Critical heat flux [kW/m^2] from 2006 LUT, convert to [W/m^2]
q_raw=np.loadtxt('2006LUTdata.txt')*1e3

In [11]:
# Convert the imported array into a (MxNxQ) where:
#   M is number of mass flux divisions
#   N is number of quality divisions
#   Q is number of pressure divisions
lenG = len(G)
lenx = len(x)
lenP = len(P)
q = np.zeros((lenG,lenx,lenP))
for i in xrange(lenG):
    for j in xrange(lenx):
        for k in xrange(lenP):
            q[i,j,k] = q_raw[i + k*lenG,j]    

In [14]:
# Create the datasets:
ds_G = sdf.Dataset('G', data=G, unit='kg/(m2.s)', is_scale=True, display_name='Mass Flux')
ds_x = sdf.Dataset('x', data=x, unit='1', is_scale=True, display_name='Quality')
ds_P = sdf.Dataset('P', data=P, unit='Pa', is_scale=True, display_name='Pressure')
ds_q = sdf.Dataset('q', data=q, unit='W/m2', scales=[ds_G,ds_x,ds_P])

In [15]:
# Create the root group and write the file:
g2 = sdf.Group('/', comment='2006 CHF LUT', datasets=[ds_G,ds_x,ds_P,ds_q])
sdf.save('2006LUT.sdf', g2)